# 🔴 **Colab Inference for ZFTurbo's [Music-Source-Separation-Training](https://github.com/ZFTurbo/Music-Source-Separation-Training)**


<font size=3>Made by [**jarredou**](https://github.com/jarredou) & **deton**</font>  
[![ko-fi](https://ko-fi.com/img/githubbutton_sm.svg)](https://ko-fi.com/Q5Q811R5YI)

Modified by **MedicDoesStuff**

In [ ]:
#@markdown #**Mount Google Drive**
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import base64
#@markdown # **Install**

%cd /content
!git clone -b colab-inference https://github.com/jarredou/Music-Source-Separation-Training

#generate new requirements.txt file for faster colab install
req = 'IyB0b3JjaCAjPT0yLjAuMQ0KbnVtcHkNCnBhbmRhcw0Kc2NpcHkNCnNvdW5kZmlsZQ0KbWxfY29sbGVjdGlvbnMNCnRxZG0NCnNlZ21lbnRhdGlvbl9tb2RlbHNfcHl0b3JjaD09MC4zLjMNCnRpbW09PTAuOS4yDQphdWRpb21lbnRhdGlvbnM9PTAuMjQuMA0KcGVkYWxib2FyZD09MC44LjENCm9tZWdhY29uZj09Mi4yLjMNCmJlYXJ0eXBlPT0wLjE0LjENCnJvdGFyeV9lbWJlZGRpbmdfdG9yY2g9PTAuMy41DQplaW5vcHM9PTAuNi4xDQpsaWJyb3NhDQpkZW11Y3MgIz09NC4wLjANCiMgdHJhbnNmb3JtZXJzPT00LjM1LjANCnRvcmNobWV0cmljcz09MC4xMS40DQpzcGFmZT09MC4zLjINCnByb3RvYnVmPT0zLjIwLjMNCnRvcmNoX2F1ZGlvbWVudGF0aW9ucw0KYXN0ZXJvaWQ9PTAuNy4wDQphdXJhbG9zcw0KdG9yY2hzZWcNCg=='
dec_req = base64.b64decode(req).decode('utf-8')
f = open("Music-Source-Separation-Training/requirements.txt", "w")
f.write(dec_req)
f.close()

!mkdir '/content/Music-Source-Separation-Training/ckpts'

print('Installing the dependencies... This will take few minutes')
!pip install --no-cache-dir --disable-pip-version-check -r 'Music-Source-Separation-Training/requirements.txt' --exists-action=w &> /dev/null

print('Installation is done !')

In [ ]:
%cd '/content/Music-Source-Separation-Training/'
import os
import torch
import yaml
from urllib.parse import quote

class IndentDumper(yaml.Dumper):
    def increase_indent(self, flow=False, indentless=False):
        return super(IndentDumper, self).increase_indent(flow, False)

def tuple_constructor(loader, node):
    # Load the sequence of values from the YAML node
    values = loader.construct_sequence(node)
    # Return a tuple constructed from the sequence
    return tuple(values)

# Register the constructor with PyYAML
yaml.SafeLoader.add_constructor('tag:yaml.org,2002:python/tuple',
tuple_constructor)

def conf_edit(config_path, chunk_size, overlap):
    with open(config_path, 'r') as f:
        data = yaml.load(f, Loader=yaml.SafeLoader)

    # handle cases where 'use_amp' is missing from config:
    if 'use_amp' not in data.keys():
      data['training']['use_amp'] = True

    data['audio']['chunk_size'] = chunk_size
    data['inference']['num_overlap'] = overlap

    if data['inference']['batch_size'] == 1:
      data['inference']['batch_size'] = 2

    print("Using custom overlap and chunk_size values:")
    print(f"overlap = {data['inference']['num_overlap']}")
    print(f"chunk_size = {data['audio']['chunk_size']}")
    print(f"batch_size = {data['inference']['batch_size']}")

    with open(config_path, 'w') as f:
        yaml.dump(data, f, default_flow_style=False, sort_keys=False, Dumper=IndentDumper, allow_unicode=True)

def download_file(url):
    # Encode the URL to handle spaces and special characters
    encoded_url = quote(url, safe=':/')

    path = 'ckpts'
    os.makedirs(path, exist_ok=True)
    filename = os.path.basename(encoded_url)
    file_path = os.path.join(path, filename)

    if os.path.exists(file_path):
        print(f"File '{filename}' already exists at '{path}'.")
        return

    try:
        response = torch.hub.download_url_to_file(encoded_url, file_path)
        print(f"File '{filename}' downloaded successfully")
    except Exception as e:
        print(f"Error downloading file '{filename}' from '{url}': {e}")

#@markdown # **Separation**
#@markdown #### Separation config:
input_folder = '/content/in' #@param {type:"string"}
output_folder = '/content/drive/MyDrive/output' #@param {type:"string"}
model = 'VOCALS-MelBand-Roformer Fullness (by Aname-Tommy)' #@param ['VOCALS-MelBand-Roformer (by Becruily)', 'INST-MelBand-Roformer (by Becruily)', 'VOCALS-MelBand-Roformer (by KimberleyJSN)', 'VOCALS-MelBand-Roformer voc_Fv3 (by Gabox)', 'VOCALS-MelBand-Roformer Fullness (by Aname-Tommy)', 'VOCALS-MelBand-Roformer Kim FT (by Unwa)', 'VOCALS-MelBand-Roformer Kim FT 2 (by Unwa)', 'VOCALS-MelBand-Roformer Kim FT 2 Bleedless (by Unwa)', 'VOCALS-Melband-Roformer BigBeta6 (by Unwa)', 'VOCALS-Melband-Roformer BigBeta5e (by Unwa)', 'VOCALS-Mel-Roformer big beta 4 (by Unwa)', 'INST-Mel-Roformer v1 (by Unwa)', 'INST-Mel-Roformer v2 (by Unwa)', 'INST-Mel-Roformer v1e (by Unwa)', 'INST-Mel-Roformer INSTV6 (by Gabox)', 'INST-Mel-Roformer INSTV6N (by Gabox)', 'KARAOKE-MelBand-Roformer (by Aufr33 & ViperX)', 'DE-REVERB-MelBand-Roformer aggr./v2/19.1729 (by Anvuew)', 'DE-REVERB-Echo-MelBand-Roformer (by Sucial)', 'DENOISE-MelBand-Roformer-2 (by Aufr33)', 'DEBLEED-MelBand-Roformer (by Unwa/97Chris)', 'VOCALS-Male Female-BS-RoFormer Male Female Beta 7_2889 (by Aufr33)']
extract_instrumental = False #@param {type:"boolean"}
export_format = 'flac PCM_16' #@param ['wav FLOAT', 'flac PCM_16', 'flac PCM_24']
use_tta = True #@param {type:"boolean"}
#@markdown ---
#@markdown *Roformers custom config:*
overlap = 2 #@param {type:"slider", min:2, max:40, step:1}
chunk_size = 485100 #@param [352800, 485100] {type:"raw"}

if export_format.startswith('flac'):
    flac_file = True
    pcm_type = export_format.split(' ')[1]
else:
    flac_file = False
    pcm_type = None

if model == 'VOCALS-MelBand-Roformer (by Becruily)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/config_vocals_becruily.yaml'
    start_check_point = 'ckpts/mel_band_roformer_vocals_becruily.ckpt'
    download_file('https://huggingface.co/becruily/mel-band-roformer-vocals/resolve/main/config_vocals_becruily.yaml')
    download_file('https://huggingface.co/becruily/mel-band-roformer-vocals/resolve/main/mel_band_roformer_vocals_becruily.ckpt')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'INST-MelBand-Roformer (by Becruily)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/config_instrumental_becruily.yaml'
    start_check_point = 'ckpts/mel_band_roformer_instrumental_becruily.ckpt'
    download_file('https://huggingface.co/becruily/mel-band-roformer-instrumental/resolve/main/config_instrumental_becruily.yaml')
    download_file('https://huggingface.co/becruily/mel-band-roformer-instrumental/resolve/main/mel_band_roformer_instrumental_becruily.ckpt')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'VOCALS-MelBand-Roformer (by KimberleyJSN)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/config_vocals_mel_band_roformer_kj.yaml'
    start_check_point = 'ckpts/MelBandRoformer.ckpt'
    download_file('https://raw.githubusercontent.com/ZFTurbo/Music-Source-Separation-Training/main/configs/KimberleyJensen/config_vocals_mel_band_roformer_kj.yaml')
    download_file('https://huggingface.co/KimberleyJSN/melbandroformer/resolve/main/MelBandRoformer.ckpt')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'VOCALS-MelBand-Roformer voc_Fv3 (by Gabox)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/voc_gabox.yaml'
    start_check_point = 'ckpts/voc_Fv3.ckpt'
    download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/vocals/voc_gabox.yaml')
    download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/vocals/voc_Fv3.ckpt')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'VOCALS-MelBand-Roformer Fullness (by Aname-Tommy)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/config.yaml'
    start_check_point = 'ckpts/FullnessVocalModel.ckpt'
    download_file('https://huggingface.co/Aname-Tommy/MelBandRoformers/resolve/main/config.yaml')
    download_file('https://huggingface.co/Aname-Tommy/MelBandRoformers/resolve/main/FullnessVocalModel.ckpt')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'VOCALS-MelBand-Roformer Kim FT (by Unwa)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/config_kimmel_unwa_ft.yaml'
    start_check_point = 'ckpts/kimmel_unwa_ft.ckpt'
    download_file('https://huggingface.co/pcunwa/Kim-Mel-Band-Roformer-FT/resolve/main/config_kimmel_unwa_ft.yaml')
    download_file('https://huggingface.co/pcunwa/Kim-Mel-Band-Roformer-FT/resolve/main/kimmel_unwa_ft.ckpt')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'VOCALS-MelBand-Roformer Kim FT 2 (by Unwa)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/config_kimmel_unwa_ft.yaml'
    start_check_point = 'ckpts/kimmel_unwa_ft2.ckpt'
    download_file('https://huggingface.co/pcunwa/Kim-Mel-Band-Roformer-FT/resolve/main/config_kimmel_unwa_ft.yaml')
    download_file('https://huggingface.co/pcunwa/Kim-Mel-Band-Roformer-FT/resolve/main/kimmel_unwa_ft2.ckpt')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'VOCALS-MelBand-Roformer Kim FT 2 Bleedless (by Unwa)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/config_kimmel_unwa_ft.yaml'
    start_check_point = 'ckpts/kimmel_unwa_ft2_bleedless.ckpt'
    download_file('https://huggingface.co/pcunwa/Kim-Mel-Band-Roformer-FT/resolve/main/config_kimmel_unwa_ft.yaml')
    download_file('https://huggingface.co/pcunwa/Kim-Mel-Band-Roformer-FT/resolve/main/kimmel_unwa_ft2_bleedless.ckpt')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'VOCALS-Melband-Roformer BigBeta6 (by Unwa)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/big_beta6.yaml'
    start_check_point = 'ckpts/big_beta6.ckpt'
    download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-big/resolve/main/big_beta6.ckpt')
    download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-big/resolve/main/big_beta6.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'VOCALS-Melband-Roformer BigBeta5e (by Unwa)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/big_beta5e.yaml'
    start_check_point = 'ckpts/big_beta5e.ckpt'
    download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-big/resolve/main/big_beta5e.ckpt')
    download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-big/resolve/main/big_beta5e.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'VOCALS-Mel-Roformer big beta 4 (by Unwa)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/config_melbandroformer_big_beta4.yaml'
    start_check_point = 'ckpts/melband_roformer_big_beta4.ckpt'
    download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-big/resolve/main/melband_roformer_big_beta4.ckpt')
    download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-big/raw/main/config_melbandroformer_big_beta4.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'INST-Mel-Roformer v1 (by Unwa)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/config_melbandroformer_inst.yaml'
    start_check_point = 'ckpts/melband_roformer_inst_v1.ckpt'
    download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-Inst/resolve/main/melband_roformer_inst_v1.ckpt')
    download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-Inst/raw/main/config_melbandroformer_inst.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'INST-Mel-Roformer v2 (by Unwa)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/config_melbandroformer_inst_v2.yaml'
    start_check_point = 'ckpts/melband_roformer_inst_v2.ckpt'
    download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-Inst/resolve/main/melband_roformer_inst_v2.ckpt')
    download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-Inst/raw/main/config_melbandroformer_inst_v2.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'INST-Mel-Roformer v1e (by Unwa)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/config_melbandroformer_inst.yaml'
    start_check_point = 'ckpts/inst_v1e.ckpt'
    download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-Inst/resolve/main/inst_v1e.ckpt')
    download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-Inst/raw/main/config_melbandroformer_inst.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'INST-Mel-Roformer INSTV6 (by Gabox)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/inst_gabox.yaml'
    start_check_point = 'ckpts/INSTV6.ckpt'
    download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/instrumental/INSTV6.ckpt')
    download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/instrumental/inst_gabox.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'INST-Mel-Roformer INSTV6N (by Gabox)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/inst_gabox.yaml'
    start_check_point = 'ckpts/INSTV6.ckpt'
    download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/instrumental/INSTV6N.ckpt')
    download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/instrumental/inst_gabox.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'KARAOKE-MelBand-Roformer (by Aufr33 & ViperX)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/config_mel_band_roformer_karaoke.yaml'
    start_check_point = 'ckpts/mel_band_roformer_karaoke_aufr33_viperx_sdr_10.1956.ckpt'
    download_file('https://huggingface.co/jarredou/aufr33-viperx-karaoke-melroformer-model/resolve/main/mel_band_roformer_karaoke_aufr33_viperx_sdr_10.1956.ckpt')
    download_file('https://huggingface.co/jarredou/aufr33-viperx-karaoke-melroformer-model/resolve/main/config_mel_band_roformer_karaoke.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'DE-REVERB-MelBand-Roformer aggr./v2/19.1729 (by Anvuew)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/dereverb_mel_band_roformer_anvuew.yaml'
    start_check_point = 'ckpts/dereverb_mel_band_roformer_anvuew_sdr_19.1729.ckpt'
    download_file('https://huggingface.co/anvuew/dereverb_mel_band_roformer/resolve/main/dereverb_mel_band_roformer_anvuew_sdr_19.1729.ckpt')
    download_file('https://huggingface.co/anvuew/dereverb_mel_band_roformer/resolve/main/dereverb_mel_band_roformer_anvuew.yaml')

elif model == 'DE-REVERB-Echo-MelBand-Roformer (by Sucial)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/config_dereverb-echo_mel_band_roformer.yaml'
    start_check_point = 'ckpts/dereverb-echo_mel_band_roformer_sdr_10.0169.ckpt'
    download_file('https://huggingface.co/Sucial/Dereverb-Echo_Mel_Band_Roformer/resolve/main/dereverb-echo_mel_band_roformer_sdr_10.0169.ckpt')
    download_file('https://huggingface.co/Sucial/Dereverb-Echo_Mel_Band_Roformer/resolve/main/config_dereverb-echo_mel_band_roformer.yaml')

elif model == 'DENOISE-MelBand-Roformer-2 (by Aufr33)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/model_mel_band_roformer_denoise.yaml'
    start_check_point = 'ckpts/denoise_mel_band_roformer_aufr33_aggr_sdr_27.9768.ckpt'
    download_file('https://huggingface.co/jarredou/aufr33_MelBand_Denoise/resolve/main/denoise_mel_band_roformer_aufr33_aggr_sdr_27.9768.ckpt')
    download_file('https://huggingface.co/jarredou/aufr33_MelBand_Denoise/resolve/main/model_mel_band_roformer_denoise.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'DEBLEED-MelBand-Roformer (by Unwa/97Chris)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/config_bleed_suppressor_v1.yaml'
    start_check_point = 'ckpts/bleed_suppressor_v1.ckpt'
    download_file('https://shared.multimedia.workers.dev/download/1/other/bleed_suppressor_v1.ckpt')
    download_file('https://shared.multimedia.workers.dev/download/1/other/config_bleed_suppressor_v1.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'VOCALS-Male Female-BS-RoFormer Male Female Beta 7_2889 (by Aufr33)':
    model_type = 'bs_roformer'
    config_path = 'ckpts/config_chorus_male_female_bs_roformer.yaml'
    start_check_point = 'ckpts/bs_roformer_male_female_by_aufr33_sdr_7.2889.ckpt'
    download_file('https://huggingface.co/RareSirMix/AIModelRehosting/resolve/main/bs_roformer_male_female_by_aufr33_sdr_7.2889.ckpt')
    download_file('https://huggingface.co/Sucial/Chorus_Male_Female_BS_Roformer/resolve/main/config_chorus_male_female_bs_roformer.yaml')
    conf_edit(config_path, chunk_size, overlap)

!python inference.py \
    --model_type {model_type} \
    --config_path '{config_path}' \
    --start_check_point '{start_check_point}' \
    --input_folder '{input_folder}' \
    --store_dir '{output_folder}' \
    {('--extract_instrumental' if extract_instrumental else '')} \
    {('--flac_file' if flac_file else '')} \
    {('--use_tta' if use_tta else '')} \
    {('--pcm_type ' + pcm_type if pcm_type else '')}

### **INST-Mel-Roformer v1/1x/2 Output File Naming Issue**  
These models have swapped output file names—files labeled as "vocals" are actually instrumentals. If you're using the **v1e** model and **uncheck** "extract_instrumentals," only one stem called **"other"** will be generated, and it will contain the instrumental.  

### **TTA (Test Time Augmentation)** - Explanation by jarredou
Enabling **TTA** will increase separation time. It slightly improves the **SDR (Signal-to-Distortion Ratio)** score, but the difference isn’t always noticeable in real-world use.  

*TTA works by running three passes on the audio file instead of one:*  
1. **Original audio**  
2. **Stereo inverted** - Left becomes right, right becomes left.
3. **Phase inverted**

The final output is an average of these passes.  

**Overlap** - higher means longer separation time. 4 is already balanced value, 2 is fast and some people still won't notice any difference. Normally there's not point going over 8.

If your separation can't start and "Total files found: 0" is shown, be aware that: <br>1) Input must be a path to a folder containing audio files, not direct path to an audio file<br> 2) The Colab is case aware - e.g. call your folder "input" not "Input".<br> 3) Check if your Google Drive mounting was executed correctly. Open file manager on the left to check if your drive folder is not empty. If it's the case, force remount with the following line:

In [ ]:
drive.mount("/content/drive", force_remount=True)